IMPORTING LIBRARIES

import requests

In [34]:
import requests #This library help us to fetch data from API
import json #This library help us to convert data into JSON format
import pandas as pd #This library help us to convert data into DataFrame format
import numpy as np #This library help us to perform mathematical operation
from sklearn.model_selection import train_test_split #This library help us to split data into train and test
from sklearn.linear_model import LinearRegression #This library help us to create Linear Regression model
from sklearn.metrics import mean_squared_error #This library help us to calculate mean absolute error
from sklearn.preprocessing import LabelEncoder #This library help us to perform scaling and normalization
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor #This library help us to create Random Forest model
from datetime import datetime,timedelta #This library help us to get current date and time
import pytz


### 2.FETCH WEATHER DATA

In [35]:
API_KEY = 'baa89485126a855a79755dea6b31d01b'
BASE_URL = 'https://api.openweathermap.org/data/2.5/' #base url for openweathermap api
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric" #contructing the API URL
    try:
        response = requests.get(url) #making a GET request to the API
        data =response.json()
    except:
      return None
    return {
       'city': data['name'],
       'current_temperature': round(data['main']['temp']),
       'feels_like':data['main']['feels_like'],
       'temp_min': round(data['main']['temp_min']),
       'temp_max': round(data['main']['temp_max']),
       'description': data['weather'][0]['description'],
       'humidity': round(data['main']['humidity']),
       'country':data['sys']['country'],
       'wind_gust_dir':data['wind']['deg'],
       'wind_gust_speed':data['wind']['speed'],
       'pressure':data['main']['pressure']
    }
get_current_weather("Nairobi")


{'city': 'Nairobi',
 'current_temperature': 19,
 'feels_like': 18.65,
 'temp_min': 19,
 'temp_max': 19,
 'description': 'scattered clouds',
 'humidity': 68,
 'country': 'KE',
 'wind_gust_dir': 160,
 'wind_gust_speed': 2.57,
 'pressure': 1020}

#### 3. Read Historical Data

In [36]:
def read_historical_data(filename):
    """Read historical data from a file"""
    df = pd.read_csv(filename) #load the file into DataFrame
    df.dropna()#remove rows with missing values
    df.drop_duplicates()
    return df

read_historical_data('weather.csv').head()

,MinTemp,MaxTemp,WindGustDir,WindGustSpeed,Humidity,Pressure,Temp,RainTomorrow
0,8.0,24.3,NW,30.0,29,1015.0,23.6,Yes
1,14.0,26.9,ENE,39.0,36,1008.4,25.7,Yes
2,13.7,23.4,NW,85.0,69,1007.2,20.2,Yes
3,13.3,15.5,NW,54.0,56,1007.0,14.1,Yes
4,7.6,16.1,SSE,50.0,49,1018.5,15.4,No


#### 4.Prepare data for training

In [37]:
def prepare_data(data):
    le = LabelEncoder()
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTommorow'] = le.fit_transform(data['RainTommorow'])

    #define the feature variable
    X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity',
       'Pressure', 'Temp']]
    y = data['RainTommorow']
    return X, y ,le #return Feature Variable and Target Variable and LabelEncoder



#### 5. **Train Rain Model**

In [38]:
def train_rain_model(X,y):
    # Define the model
    X_train ,X_test, y_train , y_test = train_test_split(X, y, test_size=0.2,random_state=42)
    model = RandomForestClassifier(n_estimators=100,random_state=42)
    model.fit(X_train, y_train) #train the model
    y_pred = model.predict(X_test)
    
    print('Mean Squared Error for Rain Model: ')
    print(mean_squared_error(y_test, y_pred))
    return model

#### 6. **Prepare Regression Data**

In [39]:
def prepare_regression_data(data,feature):
    X,y = [],[] #initialize list feature
    for i in range(len(data) -1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i+1])

    X = np.array(X).reshape(-1,1)
    y = np.array(y)
    return X,y


#### 7. **Train a Regression Model**

In [40]:
def train_regression_model(X,y):
    model = RandomForestRegressor(n_estimators=100,random_state=42)
    model.fit(X,y)
    return model




#### 8.**Predict Future**

In [41]:
def predict_future(model,current_value):
    # predict future value
    predictions = [current_value]
    for i in range(1, 10):
        next_value = model.predict(np.array[[predictions[-1]]])
        predictions.append(next_value[0])
        return predictions[1:]


9. #### **Weather Analysis**

In [43]:
def weather_view():
    city = input("Enter your city name: ")
    current_weather = get_current_weather(city)
    
    #load historical data
    historical_data =read_historical_data('weather.csv')

    #prepare and and Train the rain prediction Model
    X,y,le = prepare_data(historical_data)

    rain_model = train_rain_model(X,y)
    # Map wind direction to compass points
    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
    ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
    ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
    ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
    ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
    ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
    ("NNW", 326.25, 348.75)
     ]

    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg <= end)
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1
    current_data = {
        'Max_temp': current_weather['temp_max'],
        'Min_temp': current_weather['temp_min'],
        'WindGustDir':compass_direction_encoded,
        'wind_gust_speed': current_weather['wind_gust_speed'],
        'Humidity': current_weather['humidity'],
        'Pressure':current_weather['pressure'],
        'Temp':current_weather['current_temperature']
    

    }
    current_df = pd.DataFrame([current_data])
    # rain Prediction
    rain_prediction = rain_model.predict(current_df)[0]
    #prepare and Train the weather prediction Model for temp AND HUMIDITY
    X_temp ,y_temp = prepare_data(historical_data, target='temp')
    X_humidity ,y_humidity = prepare_data(historical_data, target='humidity')
    temp_model = train_regression_model(X_temp,y_temp)
    humidity_model = train_regression_model(X_humidity,y_humidity)

    #predict future temp and humidity
    future_temp = predict_future(temp_model,current_weather['temp_min'])
    future_humidity = predict_future(humidity_model,current_weather['humidity'])

    #prepare time for future weather
    timezone = pytz.timezone('Kenya/Nairobi')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0,second=0,microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(10)]

    #Display Results
    print(f"City: {city},{current_weather['country']}")
    print(f"Feels like: {current_weather['feel_like']}")
    print(f"Wind speed: {current_weather['wind_gust_speed']}")
    print(f"Pressure: {current_weather['pressure']}")
    print(f"Current temperature: {current_weather['current_temperature']}C")
    print(f"Maximum temperature: {current_weather['temp_max']}C")
    print(f"Minimum temperature: {current_weather['temp_min']}C")
    print(f"Current humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction : {current_weather['description']}")
    print(f"Rain Prediction : {'Yes' if rain_prediction else 'No'}")

    print(f"\n Future Prediction:")
    for time ,temp in zip(future_times,future_temp):
        print(f"{time}: {round(temp , 1)}C")
    for time ,humidity in zip(future_times,future_humidity):
        print(f"{time}: {round(humidity , 1)}%")
weather_view()


KeyError: 'RainTommorow'